In [48]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import re

events = [
    "Riding",
    "Fighting",
    "Playing",
    "Running",
    "Lying",
    "Chasing",
    "Jumping",
    "Falling",
    "Guiding",
    "Stealing",
    "Littering",
    "Tripping",
    "Pickpockering",
]
description = [
    "a person riding a bicycle on the street",  # Added context
    "multiple people engaged in a physical fight",  # More specific than "fighting"
    "a group of people playing a sport together",  # Added "sport" for visual clarity
    "a person running",  # Added context
    "a person lying motionless on the ground",  # "Motionless" helps distinguish from falling
    "a person aggressively chasing another person",  # "Aggressively" adds distinction
    "a person jumping high in the air with both feet",  # More specific than just "jumping"
    "a person accidentally falling to the ground",  # "Accidentally" helps distinguish
    "a person gently guiding another person by the arm",  # Added detail
    "a person stealing other person",  # More specific than "stealing"
    "a person deliberately throwing garbage on the ground",  # "Deliberately" adds clarity
    "a person tripping over an obstacle",  # More descriptive
    "a person pickpocketing a wallet from someone's pocket",  # Very specific
]

In [49]:
def check_metrics(scores, gt, threshold):
    tp, fp, fn, tn = 0, 0, 0, 0
    for i in range(len(scores)):
        if gt[i] ==1:
            print(f"GT: {gt[i]}, Score: {scores[i]}")
        scores[i] = 1 if scores[i] > threshold else 0
        tp += np.sum((scores[i] == 1) & (gt[i] == 1))
        fp += np.sum((scores[i] == 1) & (gt[i] == 0))  
        fn += np.sum((scores[i] == 0) & (gt[i] == 1))
        tn += np.sum((scores[i] == 0) & (gt[i] == 0))
    return tp, fp, fn, tn
rute_stored='/home/ubuntu/Tesis/Storage/Score_RulesOnebyOne'
files=os.listdir(rute_stored)
files = [f for f in files if f.endswith('.npy')]
df= pd.DataFrame(columns=['Name', 'Event', 'Mode', 'TP', 'FP', 'FN', 'TN'])
class_to_idx = {cls: idx for idx, cls in enumerate(description)}
# Example: {"a normal view...": 0, "a person riding...": 1, ...}
num_classes = len(description)  # 14
cm_sum = np.zeros((num_classes, num_classes), dtype=int)
df= pd.DataFrame(columns=['Name', 'Event', 'Mode', 'TP', 'FP', 'FN', 'TN'])
threshold = 0.05  # Define your threshold here

for file in files:
    name= file.split('_CLIP_')[0]
    mode=int(file.split('_CLIP_')[1][0])
    event=file.split('_CLIP_')[1][2:].split('.')[0]
    info_stored=np.load(f"{rute_stored}/{file}", allow_pickle=True)
    frames_number=info_stored[0]
    scores=info_stored[1]
    gt=info_stored[2]
    tp, fp, fn, tn=check_metrics(scores, gt, threshold)
    row = {
    'Name':name, 'Event':event, 'Mode':mode, 'TP':tp, 'FP':fp, 'FN':fn, 'TN':tn
    }
    df =pd.concat([df, pd.DataFrame([row])], ignore_index=True) 
print(df)


GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, Score: 0
GT: 1.0, S

In [50]:
df_respaldo=df.copy()
#df_respaldo=df_respaldo.loc[~df_respaldo["Name"].str.endswith("_1")] 
print(df_respaldo)
df_sum = df_respaldo.groupby(["Event", "Mode"])[
    ["TP", "FP", "FN", "TN"]
].sum().reset_index()

# Initialize precision, recall, accuracy with zeros
df_sum["Precision"] = 0.0
df_sum["Recall"] = 0.0
df_sum["Accuracy"] = 0.0

# Calculate only where the denominator is not zero
mask_prec = (df_sum["TP"] + df_sum["FP"]) != 0
df_sum.loc[mask_prec, "Precision"] = (
    df_sum.loc[mask_prec, "TP"] / (df_sum.loc[mask_prec, "TP"] + df_sum.loc[mask_prec, "FP"])
)

mask_recall = (df_sum["TP"] + df_sum["FN"]) != 0
df_sum.loc[mask_recall, "Recall"] = (
    df_sum.loc[mask_recall, "TP"] / (df_sum.loc[mask_recall, "TP"] + df_sum.loc[mask_recall, "FN"])
)

mask_acc = (df_sum["TP"] + df_sum["FP"] + df_sum["FN"] + df_sum["TN"]) != 0
df_sum.loc[mask_acc, "Accuracy"] = (
    (df_sum.loc[mask_acc, "TP"] + df_sum.loc[mask_acc, "TN"]) /
    (df_sum.loc[mask_acc, "TP"] + df_sum.loc[mask_acc, "FP"] + df_sum.loc[mask_acc, "FN"] + df_sum.loc[mask_acc, "TN"])
)
df_sum.drop(columns=['TP', 'FP', 'FN', 'TN'], inplace=True)
df_sum['F1'] = 2 * df_sum['Precision'].astype(float) * df_sum['Recall'].astype(float) / (df_sum['Precision'].astype(float) + df_sum['Recall'].astype(float))
df_sum['F1'].fillna(0, inplace=True)  # Handle division by zero
df_sum.drop(columns=['Event'], inplace=True)
df_sum=df_sum.groupby(["Mode"]).mean()
mode_names = {
        0: "CLIP y Reglas",
        1: "CLIP y Reglas con MLLM",
        2: "CLIP y Reglas Una clase",
    }
df_sum.rename(index=mode_names, inplace=True)
print(df_sum)

        Name                                    Event Mode  TP  FP   FN   TN
0    4_087_1                         a person running    4   0   0   53   42
1    1_096_1                         a person running    4   0   0   19   40
2     000326                a person riding a bicycle    4  27   0   32  104
3    D043_01     a certain number of persons fighting    4  56  43    5  326
4    2_093_1               a group of persons playing    4  21   4  239  126
..       ...                                      ...  ...  ..  ..  ...  ...
247  2_077_1            a person chasing other person    4  13   1   21   37
248  3_093_1                         a person running    4  13   0   54  142
249  D149_03  a person stealing other person's pocket    4   6   1   72  266
250  3_088_1     a person throwing trash in the floor    4   4   6   11   98
251  3_104_1                         a person running    4   0   0   68   81

[252 rows x 7 columns]
     Precision    Recall  Accuracy        F1
Mode   

/tmp/ipykernel_31020/2034563803.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.6834692364714603 0.8880909634949132 0.9393939393939394 0.75
 0.9131578947368421 0.21212121212121213 0.80625 0.9962168978562421
 0.9135802469135802 0.7333333333333333 0.2721518987341772
 0.37718120805369126 0.8641975308641975]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sum.loc[mask_prec, "Precision"] = (
/tmp/ipykernel_31020/2034563803.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.6236753100338218 0.2178508514386377 0.043275942298743604
 0.012257405515832482 0.5438871473354232 0.07 0.07413793103448275
 0.28884826325411334 0.16342756183745583 0.034375 0.1365079365079365
 0.5966029723991507 0.2702702702702703]' has dtype incompatible with float64, please explicitly cast to a com

In [51]:
def calculate_ap(precision, recall):
    # Sort by recall (ascending)
    sorted_indices = np.argsort(recall)
    precision = np.array(precision)[sorted_indices]
    recall = np.array(recall)[sorted_indices]

    # Pad with (0,0) and (1,0)
    precision = np.concatenate(([0], precision, [0]))
    recall = np.concatenate(([0], recall, [1]))

    # Compute AP as the area under the raw curve (no interpolation)
    ap = 0.0
    for i in range(1, len(recall)):
        delta_recall = recall[i] - recall[i - 1]
        ap += delta_recall * precision[i]

    return ap
df_check = df_respaldo.copy()

# Initialize precision, recall, accuracy with zeros
df_check["Precision"] = 0.0
df_check["Recall"] = 0.0
df_check["Accuracy"] = 0.0

# Calculate only where the denominator is not zero
mask_prec = (df_check["TP"] + df_check["FP"]) != 0
df_check.loc[mask_prec, "Precision"] = (
    df_check.loc[mask_prec, "TP"] / (df_check.loc[mask_prec, "TP"] + df_check.loc[mask_prec, "FP"])
)

mask_recall = (df_check["TP"] + df_check["FN"]) != 0
df_check.loc[mask_recall, "Recall"] = (
    df_check.loc[mask_recall, "TP"] / (df_check.loc[mask_recall, "TP"] + df_check.loc[mask_recall, "FN"])
)

mask_acc = (df_check["TP"] + df_check["FP"] + df_check["FN"] + df_check["TN"]) != 0
df_check.loc[mask_acc, "Accuracy"] = (
    (df_check.loc[mask_acc, "TP"] + df_check.loc[mask_acc, "TN"]) /
    (df_check.loc[mask_acc, "TP"] + df_check.loc[mask_acc, "FP"] + df_check.loc[mask_acc, "FN"] + df_check.loc[mask_acc, "TN"])
)
df_check['F1'] = 2 * df_check['Precision'].astype(float) * df_check['Recall'].astype(float) / (df_check['Precision'].astype(float) + df_check['Recall'].astype(float))
df_check['F1'].fillna(0, inplace=True)  # Handle division by zero
categories = df_check["Event"].unique()
# Separate rows by category
df_check = df_check.copy()
for column in df_check.select_dtypes(include=['category']).columns:
    df_check[column] = df_check[column].astype(str)
df_check.fillna(0, inplace=True)
category_dfs = {category: df_check[df_check["Event"] == category] for category in categories}

mAP_process = []
for i in range(len(categories)):
    df1 = category_dfs[categories[i]]
    #
    grouped = df1.groupby("Mode")
    # ----------------------------------------------------------------------
    # Ejecución del código
    ap_values = {}
    for mode, group in grouped:
        precision = np.array(group["Precision"].values)
        recall = np.array(group["Recall"].values)
        #print("Mode:", mode, "\n")
        #print("Precision:", precision, "Recall:", recall)
        # Comenta la siguiente línea para verificar si el error es aquí
        ap = calculate_ap(precision, recall)
        ap_values[mode] = ap
        #print(f'{mode} {categories[i]} AP: {ap:.4f}')
    mean_values = grouped[["Precision", "Recall",  "F1"]].mean()
    mean_values["AP"] = [ap_values[mode] for mode in mean_values.index]
    mean_values = mean_values[["AP"]]
    mAP_process.append(mean_values)
    # Plot the results
    mode_names = {
        0: "CLIP y Reglas",
        1: "CLIP y Reglas con MLLM",
        2: "CLIP y Reglas Una clase",
    }
    mean_values.rename(index=mode_names, inplace=True)
    # plt.xlim(0.4)
# Calculate the mean Average Precision (mAP) for each mode
mAP_values2 = pd.concat(mAP_process).groupby(level=0).mean()
print(mAP_values2)

            AP
Mode          
4     0.480005


/tmp/ipykernel_31020/4034585013.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.0 0.5656565656565656 0.84 0.6111111111111112 0.2441860465116279 1.0 1.0
 0.9887640449438202 1.0 0.6666666666666666 0.4482758620689655
 0.3333333333333333 0.7407407407407407 1.0 0.0 1.0 0.6785714285714286 1.0
 1.0 0.7102272727272727 0.9166666666666666 0.6666666666666666
 0.8378378378378378 1.0 0.9423076923076923 0.628099173553719
 0.30985915492957744 1.0 1.0 0.0 1.0 0.7047970479704797 1.0 0.4 1.0 1.0
 1.0 0.0 1.0 1.0 1.0 0.5 1.0 1.0 1.0 1.0 0.975609756097561 1.0 1.0
 0.34615384615384615 0.0 0.18604651162790697 1.0 1.0 1.0 0.02 1.0 1.0 1.0
 0.7243243243243244 1.0 1.0 1.0 1.0 0.8666666666666667 0.65625
 0.5555555555555556 0.8888888888888888 1.0 0.6666666666666666 1.0 1.0 1.0
 1.0 1.0 0.8181818181818182 0.9393939393939394 1.0 0.6666666666666666
 0.9038461538461539 1.0 1.0 0.576271186440678 0.5862068965517241 0.5 1.0
 0.66

In [52]:
df_fps= pd.read_csv("/home/ubuntu/Tesis/Results/TestingScoreDMC.csv")
print(df_fps["Process time"].sum()/(60*60), "hours\n\n")
df_fps["Process time"] = df_fps["Process time"] / df_fps["Duration"]
df_fps.loc[df_fps["Name"].str.contains("_1.mp4"), "Process time"] = (
        30.0 / df_fps["Process time"]
    )
df_fps.loc[~df_fps["Name"].str.contains("_1.mp4"), "Process time"] = (
        25.0 / df_fps["Process time"]
    )
categories = df_fps["True Event"].unique()
category_dfs = {category: df_fps[df_fps["True Event"] == category] for category in categories}
print(categories)
mAP_process = []
for i in range(len(categories)):
    df1 = category_dfs[categories[i]]
    grouped = df1.groupby("Mode")
    mean_values = grouped[["Process time"]].mean()
    mean_values = mean_values[["Process time"]]
    mAP_process.append(mean_values)
# Calculate the mean Average Precision (mAP) for each mode
mAP_values = pd.concat(mAP_process).groupby(level=0).mean()
print("\n\n", mAP_values)

2.1022290421856775 hours


['a person riding a bicycle' 'a certain number of persons fighting'
 'a group of persons playing' 'a person running'
 'a person lying in the floor' 'a person chasing other person'
 'a person jumping' 'a person falling' 'a person guiding other person'
 'a person stealing other person' 'a person throwing trash in the floor'
 'a person tripping' "a person stealing other person's pocket"]


       Process time
Mode              
4        32.399355
